In [ ]:
from pathlib import Path
import pandas as pd
import os
import csv
import numpy as np
import requests
import time
import io
import yfinance as yf
import warnings
# Initial imports
#%tensorflow_version 1.x
import tensorflow
import mibian
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lstm
import psutil
psutil.virtual_memory()

In [ ]:
##This is a place holder, the real dataframe we load will have the signals already implemented
df.to_csv('tech_AAPL_signal_ready.csv')

In [74]:
df = pd.read_pickle('tech_AAPL_signal_ready.pickle')
df = map_exp(df)
df['xp']

2018-10-15    NaN
2018-10-15    NaN
2018-10-15    NaN
2018-10-15    NaN
2018-10-15    NaN
             ... 
2019-12-18    NaN
2019-12-18    NaN
2019-12-18    NaN
2019-12-18    NaN
2019-12-18    NaN
Name: xp, Length: 3044464, dtype: object

In [75]:
## Can you figure this out?

#fix this so that the .nunique() prints out more unique values of the column
fy = df
fy['profit'] = fy['AskPrice'].astype(float) - fy['xp'].astype(float)
fy['profit'].nunique()



1

In [76]:
def test_reducer(df):
    df = df.drop(colums# all unneded columns should be dropped)

SyntaxError: unexpected EOF while parsing (<ipython-input-76-0cb47a98034b>, line 2)

In [ ]:
df = test_reducer(df)
df = select_df(df)

In [ ]:
calls = create_call_targets(df)
calls

In [ ]:
### Change %out an absolute value, put signals out of the money are going to be negatives
### Or create multipliers of -1 for put targets

##

#atm = at the money
#ctm = close to the money
#otm = out of the money
#ftm = further out of the money
#xf = expire far
#xm = expiring medium's distance
#xs = expiring soon

atm = .07
ctm = .1
otm = .15
ftm = .2
xf = 28
xm = 19
xs = 7
def create_call_targets(df): #, atm, ctm, otm, xf, xm, xs
    df = select_df(df, 'call')
    df = map_exp(df)
    df['atm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > -.01) & (df['% out'] < atm)
    df['ctm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > atm) & (df['% out'] < ctm)
    df['otm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > ctm) & (df['% out'] < otm)
    df['ftm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > otm) & (df['% out'] < ftm)
    df['atm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > 0) & (df['% out'] < atm)
    df['ctm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > atm) & (df['% out'] < ctm)
    df['otm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > ctm) & (df['% out'] < otm)
    df['ftm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > otm) & (df['% out'] < ftm)
    df['atm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > 0) & (df['% out'] < atm)
    df['ctm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > atm) & (df['% out'] < ctm)
    df['otm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > ctm) & (df['% out'] < otm)
    return df

def create_put_targets(df, atm, ctm, otm, xf, xm, xs):
    df['atm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > .01) & (df['% out'] < (-1 * atm))
    df['ctm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > (-1 * atm)) & (df['% out'] < (-1 * ctm))
    df['otm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > (-1 * ctm)) & (df['% out'] < (-1 * otm))
    df['ftm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > (-1 * otm)) & (df['% out'] < (-1 * ftm))
    df['atm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] >  0) & (df['% out'] < (-1 * atm))
    df['ctm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > (-1 * atm)) & (df['% out'] < (-1 * ctm))
    df['otm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > (-1 * ctm)) & (df['% out'] < (-1 * otm))
    df['ftm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > (-1 * otm)) & (df['% out'] < (-1 * ftm))
    df['atm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > (-1 * 0) )& (df['% out'] <(-1 *  atm))
    df['ctm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > (-1 * atm)) & (df['% out'] < (-1 * ctm))
    df['otm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > (-1 * ctm)) & (df['% out'] < (-1 * otm))
    return df

def select_df(df, option): #symbol choice
    df = df.loc[df['PutCall'] == option]
    #df = df.loc[df['Symbol'] == symbol]
    return df

def map_exp(df):
    xp = df
    xp.loc[xp['exp?']== True, 'xp' ] = xp.loc[df['exp?'] == True, 'AskPrice']
    xp = xp.loc[xp['exp?']== True]
    xp= xp[['optionkey', 'xp']]
    df['xp']=df.optionkey.map(xp.set_index('optionkey')['xp'])
    return df


def create_spreads(data):
    #Sort Values and shift to set up collateral requirements
    data['DateKey']=data['DataDate'].astype('str')+data['ExpirationDate'].astype('str') 
    data.groupby('DateKey') 
    data.sort_values('DataDate') 
    data['Prev_Strike']=data['StrikePrice'].shift(1)
    data['Prev_AskPrice']=data['AskPrice'].shift(1)
    data['Prev_DateKey']=data['DateKey'].shift(1)

    # Create nan values where the shifted value uses a different day
    for i in data.index:
        if data.loc[i, 'Prev_DateKey']!=data.loc[i, 'DateKey']:
            data.loc[i, 'Prev_Strike']=np.nan
            data.loc[i, 'Prev_AskPrice']=np.nan
            
    return data

In [ ]:
#
def prepare_test(df):
    ##using map exp function already built
    df = map_exp(df)
    ## select options or puts df
    df = select_df(df, symbol, option)
    #create spread for potential spread strategy
    df = create_spread(df)

In [ ]:

def testing(df):
    
    bl_1_LTSM = df.loc[(df['bl_1']== 1) & (df['LSTM'] == 1)]
    bl_1_LTSM['log'] = bl_1_LTSM['AskPrice'].astype(float) - bl_1_LTSM['BidPrice'].astype(float)
    bl_1_LTSM_total = bl_1_LTSM['log'].sum()
#     bl_1_LTSM = short_close['log'].sum()
    bl_1_LTSM_log = bl_1_LTSM['log']
#     bl_1_LTSM_log_df = pd.DataFrame(bl_1_LTSM, columns = ['log'])
    bl_1_LTSM['log'].to_csv('bl_1_LTSM.csv')
    df['bl_1_LTSM_log'] = bl_1_LTSM['log']
    return df


In [ ]:
def basic_signals(df):
    #define emoty series for br and bl
    br_1 = []
    br_2 = []
    br_3 = []
    br_4 = []
    bl_1 = []
    bl_2 = []
    bl_3 = []
    bl_4 = []
    for index, row in df.iterrows():
        # Put signals and bollinger_signal -1 determines bearish signal
        if (row['Put signals'] == True and row['bollinger_signal'] == -1):
            br_1.append(True)
        else:
            br_1.append(False)
        # Put signals and relative volatility 0 determines bearish signal
        if (row['Put signals'] == True and row['rvol signal'] == 0):
            br_2.append(True)
        else:
            br_2.append(False)
        #Put/call signal flashing bearish and High Underlying Vol SD
        if (row['Put signals'] == True and row['rVol SD'] > 1.4):
            br_3.append(True)
        else:
            br_3.append(False)
        #Put/call signal flashing bearish, high Option rVol SD and delta
        if (row['Put'] == True and row ['Option rvol delta'] > 1.4 and row['Option rVol SD'] > 1.4):
            br_4.append(True)
        else:
            br_4.append(False)
        
        
        #high volatility + a put/call signal could identify stock movement
        if (row['rvol signal'] == 1 and row['Call signals'] == True):
            bl_1.append(True)
        else:
            bl_1.append(False)
        #Put/call signal and Option rvol signals a bullish movement
        if (row['Call signals'] == True and row['Option rVol'] > 1.4):
            bl_2.append(True)
        else:
            bl_2.append(False)
        #Put/call signal flashing Bullish and High Underlying Vol SD
        if (row['Call signals'] == True and row['rVol SD'] > 1.4):
            bl_3.append(True)
        else:
            bl_3.append(False)
        #Put/call signal flashing Bullish and bollinger signal = 1 indicates possible bullish movement
        if (row['Call signals'] == True and row['bollinger_signal'] = 1):
            bl_4.append(True)
        else:
            bl_4.append(False)
    #append series columns to dataframe
    df['br_1'] = br_1
    df['br_2'] = br_2
    df['br_3'] = br_3
    df['br_4'] = br_4
    df['bl_1'] = bl_1
    df['bl_2'] = bl_2
    df['bl_3'] = bl_3
    df['bl_4'] = bl_4
    return df

In [ ]:
 def final_signals(df):
    #define emoty series for br and bl
    #Create signal columns that will enter us in to a strategy
#br columns will be bearish signals
#bl columns will be bullish signals
#Bearish signals
  # br_1 - Put signals = True & bollinger_signal = -1
  # br_2 - Put signals = True and rvol_signal = 0
  # br_3 - Percentage of puts > 0.65 and Option rVol SD > 1.5
  # br_4 - option rvol delta > 0.3 and Put signals = True
  # br_5 - volume delta > 0.6 and Put signals = True
  # br_6 - rvol SD > 1.4 and Put signals = True and rvol signal = 1
#Bullish signals
  # bl_1 - rvol signal = 1 & Call signals = True
  # bl_2 - Call signals = True and Option rVol > 1.4
  # bl_3 - Percentage of calls > 0.65 and Call signals = True Option rVol SD > 1.5
  # bl_4 - option rvol delta > 0.3 and Call signals = True
  # bl_5 - volume delta > 0.6 and Call signals = True
  # bl_6 - rvol SD > 1.4 and Call signals = True and rvol signal = 1
def final_signals(df):
    #define empty series for br and bl 
    br_1 = []
    br_2 = []
    br_3 = []
    br_4 = []
    br_5 = []
    br_6 = []
    
    bl_1 = []
    bl_2 = []
    bl_3 = []
    bl_4 = []
    bl_5 = []
    bl_6 = []
    
    for index, row in df.iterrows():
        
        # Put signals and bollinger_signal -1 determines bearish signal
        if (row['Put signals'] == True & row['bollinger_signal'] == -1):
            br_1.append(True)
        else:
            br_1.append(False)
        # Put signals and relative volatility 0 determines bearish signal
        if (row['Put signals'] == True & row['rvol signal'] == 0):
            br_2.append(True)
        else:
            br_2.append(False)
        if (row['Percent of Put'] > 0.65 and row['Option rVol SD'] > 1.5):
            br_3.append(True)
        else:
            br_3.append(False)
        if (row['option rvol delta'] > 0.3 and row['Put signals'] == True):
            br_4.append(True)
        else:
            br_4.append(False)
        if (row['volume delta'] > 0.0 and row['Put signals'] == True):
            br_5.append(True)
        else:
            br_5.append(False)
        if (row['rvol SD'] > 1.4 and row['Put signals'] == True & row['rvol signal'] == 1):
            br_6.append(True)
        else:
            br_6.append(False)
            
        #high volatility + a put/call signal could identify stock movement
        if (row['rvol signal'] == 1 & row['Call signals'] == True):
            bl_1.append(True)
        else:
            bl_1.append(False)
            
        #Put/call signal and Option rvol signals a bullish movement
        if (row['Call signals'] == True and row['Option rVol'] > 1.4):
            bl_2.append(True)
        else:
            bl_2.append(False)
        if (row['Call signals'] == True and row['Percent of calls'] > 0.65 and row['Option rVol SD'] > 1.5):
            bl_3.append(True)
        else:
            bl_3.append(False)
        if (row['Call signals'] == True and row['option rvol delta'] > 0.3):
            bl_4.append(True)
        else:
            bl_4.append(False)
        if (row['Call signals'] == True and row['volume delta'] > 0.6):
            bl_5.append(True)
        else:
            bl_5.append(False)
        if (row['Call signals'] == True and row['rvol SD'] > 1.4 and row['rvol signal'] == 1):
            bl_6.append(True)
        else:
            bl_6.append(False)                                                                    
    #append series columns to dataframe
    df['br_1'] = br_1
    df['br_2'] = br_2
    df['br_3'] = br_3
    df['br_4'] = br_4
    df['br_5'] = br_5
    df['br_6'] = br_6
    
    df['bl_1'] = bl_1
    df['bl_2'] = bl_2
    df['bl_3'] = bl_3
    df['bl_4'] = bl_4
    df['bl_5'] = bl_5
    df['bl_6'] = bl_6
     
    return df

In [ ]:
#### #### Start appending values and creating csvs
### Do this for every variation

# symbol ask  xp boop signal  boop log
# tsla   5    7    1              -2
# pd. read_csv(tech)
# pd['LSTM'] = dummies
   
#add every test to a master frame?
mf = {}
def sell_testing(df):
    bl_1_LTSM = df.loc[(df['bl_1']== True) & (df['LSTM'] == 1)]
    bl_1_LTSM['log'] = bl_1_LTSM['AskPrice'] - bl_1_LTSM['xp']
    bl_1_LTSM_log = bl_1_LTSM['log']
    bl_1_LTSM_log.to_csv('bl_1_LTSM.csv')
    
    bl_2_LTSM = df.loc[(df['bl_2']== True) & (df['LSTM'] == 1)]
    bl_2_LTSM['log'] = bl_2_LTSM['AskPrice'] - bl_2_LTSM['xp']
    bl_2_LTSM_total = bl_2_LTSM['log'].sum()
    bl_2_LTSM_log = bl_2_LTSM['log']
    bl_2_LTSM_log.to_csv('bl_2_LTSM.csv')
    
    bl_3_LTSM = df.loc[(df['bl_3']== True) & (df['LSTM'] == 1)]
    bl_3_LTSM['log'] = bl_3_LTSM['AskPrice'] - bl_3_LTSM['xp']
    bl_3_LTSM_total = bl_3_LTSM['log'].sum()
    bl_3_LTSM_log = bl_3_LTSM['log']
    bl_3_LTSM_log.to_csv('bl_3_LTSM.csv')
    
    bl_4_LTSM = df.loc[(df['bl_4']== True) & (df['LSTM'] == 1)]
    bl_4_LTSM['log'] = bl_4_LTSM['AskPrice'] - bl_4_LTSM['xp']
    bl_4_LTSM_total = bl_4_LTSM['log'].sum()
    bl_4_LTSM_log = bl_4_LTSM['log']
    bl_4_LTSM_log.to_csv('bl_4_LTSM.csv')
    
    bl_5_LTSM = df.loc[(df['bl_5']== True) & (df['LSTM'] == 1)]
    bl_5_LTSM['log'] = bl_5_LTSM['AskPrice'] - bl_5_LTSM['xp']
    bl_5_LTSM_total = bl_5_LTSM['log'].sum()
    bl_5_LTSM_log = bl_5_LTSM['log']
    bl_5_LTSM_log.to_csv('bl_5_LTSM.csv')
    
    bl_6_LTSM = df.loc[(df['bl_6']== True) & (df['LSTM'] == 1)]
    bl_6_LTSM['log'] = bl_6_LTSM['AskPrice'] - bl_6_LTSM['xp']
    bl_6_LTSM_total = bl_6_LTSM['log'].sum()
    bl_6_LTSM_log = bl_6_LTSM['log']
    bl_6_LTSM_log.to_csv('bl_6_LTSM.csv')
    
    bl_1_rf = df.loc[(df['bl_1']== True) & (df['RF'] == 1)]
    bl_1_rf['log'] = bl_1_rf['AskPrice'] - bl_1_rf['xp']
    bl_1_rf_log = bl_1_rf['log']
    bl_1_rf_log.to_csv('bl_1_rf.csv')
    
    bl_2_rf = df.loc[(df['bl_2']== True) & (df['RF'] == 1)]
    bl_2_rf['log'] = bl_2_rf['AskPrice'] - bl_2_rf['xp']
    bl_2_rf_total = bl_2_rf['log'].sum()
    bl_2_rf_log = bl_2_rf['log']
    bl_2_rf_log.to_csv('bl_2_rfrf.csv')
    
    bl_3_rf = df.loc[(df['bl_3']== True) & (df['RF'] == 1)]
    bl_3_rf['log'] = bl_3_rf['AskPrice'] - bl_3_rf['xp']
    bl_3_rf_total = bl_3_rf['log'].sum()
    bl_3_rf_log = bl_3_rf['log']
    bl_3_rf_log.to_csv('bl_3_rf.csv')
    
    bl_4_rf = df.loc[(df['bl_4']== True) & (df['RF'] == 1)]
    bl_4_rf['log'] = bl_4_rf['AskPrice'] - bl_4_rf['xp']
    bl_4_rf_total = bl_4_rf['log'].sum()
    bl_4_rf_log = bl_4_rf['log']
    bl_4_rf_log.to_csv('bl_4_rf.csv')
    
    bl_5_rf = df.loc[(df['bl_5']== True) & (df['RF'] == 1)]
    bl_5_rf[rf'log'] = bl_5_rf['AskPrice'] - bl_5_rf['xp']
    bl_5_rf_total = bl_5_rf'log'].sum()
    bl_5_rf_log = bl_5_rf['log']
    bl_5_rf_log.to_csv('bl_5_rf.csv')
    
    bl_6_rf = df.loc[(df['bl_6']== True) & (df['RF'] == 1)]
    bl_6_rf['log'] = bl_6_rf['AskPrice'] - bl_6_rf['xp']
    bl_6_rf_total = bl_6_rf['log'].sum()
    bl_6_rf_log = bl_6_rf['log']
    bl_6_rf_log.to_csv('bl_6_rf.csv')
    
    return df

    
def buy_testing(df):
    bl_1_LTSM = df.loc[(df['bl_1']== True) & (df['LSTM'] == 1)]
    bl_1_LTSM['log'] = bl_1_LTSM['xp'] - bl_1_LTSM['AskPrice'] 
    bl_1_LTSM_total = bl_1_LTSM['log'].sum()
    bl_1_LTSM_log = bl_1_LTSM['log']
    bl_1_LTSM_log.to_csv('bl_1_LTSM.csv')

    
    bl_2_LTSM = df.loc[(df['bl_2']== True) & (df['LSTM'] == 1)]
    bl_2_LTSM['log'] = bl_2_LTSM['AskPrice'] - bl_2_LTSM['xp']
    bl_2_LTSM_total = bl_2_LTSM['log'].sum()
    bl_2_LTSM_log = bl_2_LTSM['log']
    bl_2_LTSM_log.to_csv('bl_2_LTSM.csv')
    
    bl_3_LTSM = df.loc[(df['bl_3']== True) & (df['LSTM'] == 1)]
    bl_3_LTSM['log'] = bl_3_LTSM['AskPrice'] - bl_3_LTSM['xp']
    bl_3_LTSM_total = bl_3_LTSM['log'].sum()
    bl_3_LTSM_log = bl_3_LTSM['log']
    bl_3_LTSM_log.to_csv('bl_3_LTSM.csv')
    
    bl_4_LTSM = df.loc[(df['bl_4']== True) & (df['LSTM'] == 1)]
    bl_4_LTSM['log'] = bl_4_LTSM['AskPrice'] - bl_4_LTSM['xp']
    bl_4_LTSM_total = bl_4_LTSM['log'].sum()
    bl_4_LTSM_log = bl_4_LTSM['log']
    bl_4_LTSM_log.to_csv('bl_4_LTSM.csv')
    
    bl_5_LTSM = df.loc[(df['bl_5']== True) & (df['LSTM'] == 1)]
    bl_5_LTSM['log'] = bl_5_LTSM['AskPrice'] - bl_5_LTSM['xp']
    bl_5_LTSM_total = bl_5_LTSM['log'].sum()
    bl_5_LTSM_log = bl_5_LTSM['log']
    bl_5_LTSM_log.to_csv('bl_5_LTSM.csv')
    
    bl_6_LTSM = df.loc[(df['bl_6']== True) & (df['LSTM'] == 1)]
    bl_6_LTSM['log'] = bl_6_LTSM['AskPrice'] - bl_6_LTSM['xp']
    bl_6_LTSM_total = bl_6_LTSM['log'].sum()
    bl_6_LTSM_log = bl_6_LTSM['log']
    bl_6_LTSM_log.to_csv('bl_6_LTSM.csv')
    
    bl_1_rf = df.loc[(df['bl_1']== True) & (df['rfLSTM'] == 1)]
    bl_1_rf['log'] = bl_1_rf['xp'] - bl_1_rf['AskPrice'] 
    bl_1_rf_total = bl_1_rf['log'].sum()
    bl_1_rf_log = bl_1_rf['log']
    bl_1_rf_log.to_csv('bl_1_rf.csv')

    
    bl_2_rf = df.loc[(df['bl_2']== True) & (df['LSTM'] == 1)]
    bl_2_rf['log'] = bl_2_rf['AskPrice'] - bl_2_rf['xp']
    bl_2_rf_total = bl_2_rf['log'].sum()
    bl_2_rf_log = bl_2_rf['log']
    bl_2_rf_log.to_csv('bl_2_rf.csv')
    
    bl_3_rf = df.loc[(df['bl_3']== True) & (df['LSTM'] == 1)]
    bl_3_rf['log'] = bl_3_rf['AskPrice'] - bl_3_rf['xp']
    bl_3_rf_total = bl_3_rf['log'].sum()
    bl_3_rf_log = bl_3_rf['log']
    bl_3_rf_log.to_csv('bl_3_rf.csv')
    
    bl_4_rf = df.loc[(df['bl_4']== True) & (df['LSTM'] == 1)]
    bl_4_rf['log'] = bl_4_rf['AskPrice'] - bl_4_rf['xp']
    bl_4_rf_total = bl_4_rf['log'].sum()
    bl_4_rf_log = bl_4_rf['log']
    bl_4_rf_log.to_csv('bl_4_rf.csv')
    
    bl_5_rf = df.loc[(df['bl_5']== True) & (df['LSTM'] == 1)]
    bl_5_rf['log'] = bl_5_rf['AskPrice'] - bl_5_rf['xp']
    bl_5_rf_total = bl_5_rf['log'].sum()
    bl_5_rf_log = bl_5_rf['log']
    bl_5_rf_log.to_csv('bl_5_rf.csv')
    
    bl_6_rf = df.loc[(df['bl_6']== True) & (df['LSTM'] == 1)]
    bl_6_rf['log'] = bl_6_rf['AskPrice'] - bl_6_rf['xp']
    bl_6_rf_total = bl_6_rf['log'].sum()
    bl_6_rf_log = bl_6_rf['log']
    bl_6_rf_log.to_csv('bl_6_rf.csv')
    
    return df


In [ ]:
#Create signal columns that will enter us in to a strategy
#br columns will be bearish signals
#bl columns will be bullish signals
#Bearish signals
  # br_1 - Put signals = True & bollinger_signal = -1
  # br_2 - Put signals = True and rvol_signal = 0
  # br_3 - Percentage of puts > 0.65 and Option rVol SD > 1.5
  # br_4 - option rvol delta > 0.3 and Put signals = True
  # br_5 - volume delta > 0.6 and Put signals = True
  # br_6 - rvol SD > 1.4 and Put signals = True and rvol signal = 1
#Bullish signals
  # bl_1 - rvol signal = 1 & Call signals = True
  # bl_2 - Call signals = True and Option rVol > 1.4
  # bl_3 - Percentage of calls > 0.65 and Call signals = True Option rVol SD > 1.5
  # bl_4 - option rvol delta > 0.3 and Call signals = True
  # bl_5 - volume delta > 0.6 and Call signals = True
  # bl_6 - rvol SD > 1.4 and Call signals = True and rvol signal = 1
def final_signals(df):
    #define empty series for br and bl 
    br_1 = []
    br_2 = []
    br_3 = []
    br_4 = []
    br_5 = []
    br_6 = []
    
    bl_1 = []
    bl_2 = []
    bl_3 = []
    bl_4 = []
    bl_5 = []
    bl_6 = []
    
    for index, row in df.iterrows():
        
        # Put signals and bollinger_signal -1 determines bearish signal
        if (row['Put signals'] == True & row['bollinger_signal'] == -1):
            br_1.append(True)
        else:
            br_1.append(False)
        # Put signals and relative volatility 0 determines bearish signal
        if (row['Put signals'] == True & row['rvol signal'] == 0):
            br_2.append(True)
        else:
            br_2.append(False)
        if (row['Percent of Put'] > 0.65 and row['Option rVol SD'] > 1.5):
            br_3.append(True)
        else:
            br_3.append(False)
        if (row['option rvol delta'] > 0.3 and row['Put signals'] == True):
            br_4.append(True)
        else:
            br_4.append(False)
        if (row['volume delta'] > 0.0 and row['Put signals'] == True):
            br_5.append(True)
        else:
            br_5.append(False)
        if (row['rvol SD'] > 1.4 and row['Put signals'] == True & row['rvol signal'] == 1):
            br_6.append(True)
        else:
            br_6.append(False)
            
        #high volatility + a put/call signal could identify stock movement
        if (row['rvol signal'] == 1 & row['Call signals'] == True):
            bl_1.append(True)
        else:
            bl_1.append(False)
            
        #Put/call signal and Option rvol signals a bullish movement
        if (row['Call signals'] == True and row['Option rVol'] > 1.4):
            bl_2.append(True)
        else:
            bl_2.append(False)
        if (row['Call signals'] == True and row['Percent of calls'] > 0.65 and row['Option rVol SD'] > 1.5):
            bl_3.append(True)
        else:
            bl_3.append(False)
        if (row['Call signals'] == True and row['option rvol delta'] > 0.3):
            bl_4.append(True)
        else:
            bl_4.append(False)
        if (row['Call signals'] == True and row['volume delta'] > 0.6):
            bl_5.append(True)
        else:
            bl_5.append(False)
        if (row['Call signals'] == True and row['rvol SD'] > 1.4 and row['rvol signal'] == 1):
            bl_6.append(True)
        else:
            bl_6.append(False)                                                                    
    #append series columns to dataframe
    df['br_1'] = br_1
    df['br_2'] = br_2
    df['br_3'] = br_3
    df['br_4'] = br_4
    df['br_5'] = br_5
    df['br_6'] = br_6
    
    df['bl_1'] = bl_1
    df['bl_2'] = bl_2
    df['bl_3'] = bl_3
    df['bl_4'] = bl_4
    df['bl_5'] = bl_5
    df['bl_6'] = bl_6
     
    return df